In [166]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

In [167]:
!cl

usage: cl [ option... ] filename... [ /link linkoption... ]


Microsoft (R) C/C++ Optimizing Compiler Version 19.43.34810 for x64
Copyright (C) Microsoft Corporation.  All rights reserved.



In [168]:
def load_numpy(filename: str) -> np.ndarray:
    return np.load(f'./numpy/{filename}')

In [169]:
vertices = load_numpy('piece_vertices.npy')
triangles = load_numpy('triangle_vertices.npy')
normals = load_numpy('triangle_normals.npy')

In [204]:
mod = SourceModule("""
__device__ float3 cross_product(float3 a, float3 b) {
    return make_float3(
        a.y * b.z - a.z * b.y,
        a.z * b.x - a.x * b.z,
        a.x * b.y - a.y * b.x
    );
}

__device__ float dot_product(float3 a, float3 b) {
    return a.x * b.x + a.y * b.y + a.z * b.z;
}

__device__ float3 subtract(float3 a, float3 b) {
    return make_float3(a.x - b.x, a.y - b.y, a.z - b.z);
}

__device__ int ray_intersects_triangle(float3 orig, float3 dir,
                                        float3 v0, float3 v1, float3 v2) {
    const float EPSILON = 1e-7f;
    float3 b;
    float v;
    float3 edge1 = subtract(v1, v0);
    float3 edge2 = subtract(v2, v0);

    float3 a = cross_product(dir, edge2);
    float det = dot_product(edge1, a);
    if (fabsf(det) < EPSILON)
        return 0;

    float3 c = subtract(orig, v0);
    float u = dot_product(a, c);
    if (det > EPSILON) {
        if (u < 0 || u > det)
            return 0;

        b = cross_product(c, edge1);
        v = dot_product(b, dir);
        if (v < 0 || u + v > det)
            return 0;
    } else {
        if (u > 0 || u < det)
            return 0;

        b = cross_product(c, edge1);
        v = dot_product(b, dir);
        if (v > 0 || u + v < det)
            return 0;
    }

    float inv_det = 1.0f / det;
    float t = inv_det * dot_product(edge2, b);

    if (t < EPSILON || t > 1 - EPSILON)
        return 0;

    return 1;
}

__global__ void point_in_mesh(float *triangles, int num_triangles,
                              float *points, int num_points,
                              bool *output_flags) {
    int pt_idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (pt_idx >= num_points) return;

    float3 query = make_float3(
        points[pt_idx * 3 + 0],
        points[pt_idx * 3 + 1],
        points[pt_idx * 3 + 2]
    );
    float3 ray_dir = make_float3(1.0f, 1.0f, 1.0f);

    int hit_count = 0;

    for (int tri_idx = 0; tri_idx < num_triangles; ++tri_idx) {
        float3 v0 = make_float3(
            triangles[tri_idx * 9 + 0],
            triangles[tri_idx * 9 + 1],
            triangles[tri_idx * 9 + 2]
        );
        float3 v1 = make_float3(
            triangles[tri_idx * 9 + 3],
            triangles[tri_idx * 9 + 4],
            triangles[tri_idx * 9 + 5]
        );
        float3 v2 = make_float3(
            triangles[tri_idx * 9 + 6],
            triangles[tri_idx * 9 + 7],
            triangles[tri_idx * 9 + 8]
        );

        hit_count += ray_intersects_triangle(query, ray_dir, v0, v1, v2);
    }

    output_flags[pt_idx] = (hit_count % 2) == 1;
}
""")

In [205]:
point_in_mesh = mod.get_function("point_in_mesh")

In [206]:
nr_triangles = np.int32(len(triangles))
nr_vertices = np.int32(len(vertices))

In [207]:
assert vertices.flatten().flags['C_CONTIGUOUS']
assert triangles.flatten().flags['C_CONTIGUOUS']

In [208]:
triangles_gpu = cuda.mem_alloc(triangles.nbytes)
vertices_gpu = cuda.mem_alloc(vertices.nbytes)
output_cpu = np.empty(len(vertices), dtype=np.bool_)
output_gpu = cuda.mem_alloc(output_cpu.nbytes)

In [209]:
cuda.memcpy_htod(triangles_gpu, triangles.flatten())
cuda.memcpy_htod(vertices_gpu, vertices.flatten())

In [210]:
threads_per_block = 64
blocks = (len(vertices) + threads_per_block - 1) // threads_per_block

In [211]:
def get_points_in_mesh():
    point_in_mesh(triangles_gpu, nr_triangles,
              vertices_gpu, nr_vertices,
              output_gpu,
              block=(threads_per_block, 1, 1), grid=(blocks, 1, 1))
    cuda.memcpy_dtoh(output_cpu, output_gpu)

In [212]:
%timeit get_points_in_mesh()

588 µs ± 40.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [213]:
get_points_in_mesh()

In [214]:
output_cpu.sum()

50

In [215]:
np.where(output_cpu)

(array([1680, 1683, 1741, 1798, 2245, 2298, 2299, 2300, 2301, 2302, 2303,
        2345, 2346, 2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355,
        2356, 2357, 2358, 2359, 2360, 2361, 2362, 2406, 2407, 2408, 2456,
        2457, 2458, 2459, 2498, 2499, 2501, 2549, 2600, 2609, 2651, 2655,
        2690, 2701, 2741, 2742, 2793, 2945], dtype=int64),)